In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Нормализация данных
def norm_a(param, param_mean, param_std):
    if param_mean == 0 and param_std == 0:
        param_mean = param.mean()
        param_std = param.std()
    print(param_mean, param_std)
    ret_val = (param - param_mean) / param_std
    return ret_val, param_mean, param_std

In [3]:
def polyfit(x, y):
    correlation = np.corrcoef(x, y)[0,1]
    return correlation**2

In [4]:
def calc_mse(y, y_pred):
    return np.mean((y - y_pred)**2)

In [5]:
# Создадим класс для нашей модели
class LineReg:
    
    def __init__(self, alpha = 1e-2, lmbd = 1e-2):
        self.alpha = alpha
        self.lmbd = lmbd
    
    def fit(self, x_train, y_tr, x_test, y_tes, iterations):
        n = x_train.shape[1]
        W = np.random.randn(x_train.shape[0])
        err_pred = np.inf
        self.err_train = []
        self.err_test = []
        self.score_train = []
        self.score_test = []
        for i in range(iterations):
            y_pr = np.dot(W, x_train)
            y_pr2 = np.dot(W, x_test)
            err = calc_mse(y_tr, y_pr)
            err2 = calc_mse(y_tes, y_pr2)
            score = polyfit(y_tr, y_pr)
            score2 = polyfit(y_tes, y_pr2)
            self.err_train.append(err)
            self.err_test.append(err2)
            self.score_train.append(score)
            self.score_test.append(score2)
            if err_pred < err:
                break
                self.W = w_pred.copy()
                return err_train, err_test, score_train, score_test
            err_pred = err
            w_pred = W.copy()
            W -= self.alpha * (1/n * 2 * np.dot((y_pr - y_tr), x_train.T) + self.lmbd * np.sign(W))
            if i % 100 == 0:
                print(i, W, err)
                print('\n')
        self.W = W.copy()
    
    def predict(self, x):
        return np.dot(self.W, x)

In [6]:
# Функция для обработки данных
def process_data(data_in, list_params, list_norm_param, norm_index, qual):
    data_id = data_in[:, 0]
    data_out = data_in[:, list_params]
    data_out = np.hstack((data_out, np.ones((data_out.shape[0], 1))))
    data_out = data_out.T
    data_out[0] = np.log(data_out[0])
    data_out[2] = data_out[2]**2
    data_out[3] = data_out[3]**2
    norm_param = []
    k = 0
    for i in list_norm_param:
        data_out[i], o, p = norm_a(data_out[i], norm_index[k][0], norm_index[k][1])
        norm_param.append([o, p])
        k = k + 1
    k = 0
    for i in data_out[qual]:
        if i == 4.0:
            data_out[qual, k] = 3.0
        k = k + 1
    return data_id, data_out, norm_param

In [7]:
#  Читаем данные из файлов
train = pd.read_csv('train.csv').values
test = pd.read_csv('test.csv').values

In [8]:
# Разбиваем тренировочные данные на тенировочную и тестовую выборки
X_tr, X_tes, y_train, y_test = train_test_split(train[:,:11], train[:,11], test_size=0.25)

In [9]:
# Обрабатываем тренировочную выборку
list_param = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
norm_param = [0, 2]
qualification = 3
norm_ind = [[0, 0], [0, 0]]
id_train, X_train, norm_ind = process_data(X_tr, list_param, norm_param, norm_ind, qualification)

3.8087224673756825 0.18194734605184845
3151791.3333333335 2047294.0163684897


In [10]:
# Обрабатываем тестовую выборку
id_test, X_test, norm_ind = process_data(X_tes, list_param, norm_param, norm_ind, qualification)

3.8087224673756825 0.18194734605184845
3151791.3333333335 2047294.0163684897


In [11]:
# Обрабатываем целиком данные из тренировочного файла 
id_all, X_all, norm_ind = process_data(train[:,:11], list_param, norm_param, norm_ind, qualification)
y_all = train[:,11] 

3.8087224673756825 0.18194734605184845
3151791.3333333335 2047294.0163684897


In [12]:
# Обрабатываем данные из тестового файла
id_test_main, X_test_main, norm_ind = process_data(test[:,:11], list_param, norm_param, norm_ind, qualification)

3.8087224673756825 0.18194734605184845
3151791.3333333335 2047294.0163684897


In [13]:
# Обучаем модель на тренировочной выборке
lng = LineReg()
lng.fit(X_train, y_train, X_test, y_test, 10001)

0 [-0.64881547  1.62426604  0.76371414  4.30770983  1.08736037 -1.53229192
 -1.99267319  0.51492883 -1.10901654  0.25429073  1.78649143] 5280.260174691303


100 [-0.7072145   4.53901996 -9.54492088  7.09768825  8.5713815   1.37920575
  0.55900839  1.73196891 -0.20921278  0.62109711 23.01711378] 250.53235644727954


200 [-0.49033841  2.78335426 -7.70659269  6.04866014  9.74288728  2.25422245
  1.44795876  2.14443652  0.22292956  0.75315517 31.64528021] 160.73453437157187


300 [-0.35897924  1.9815009  -4.89108855  5.10841783  9.60461432  2.58571356
  1.8850104   2.34253846  0.504287    0.82999348 37.19335623] 119.43314018691993


400 [-0.28158725  1.47557826 -2.58676581  4.38977799  9.09905052  2.66866035
  2.10189051  2.43641499  0.70569444  0.87948615 41.24963495] 96.2683640194281


500 [-0.22907882  1.11447872 -0.85187192  3.85439102  8.50839946  2.6157001
  2.18711101  2.46544314  0.85217755  0.91087678 44.32598998] 82.85431962172673


600 [-0.19070397  0.84641459  0.43413055  3.456

In [14]:
# Предсказываем значения для каждого набора данных, выводим метрику качества по каждому набору
y_train_pred = lng.predict(X_train) 
y_test_pred = lng.predict(X_test) 
y_all_pred = lng.predict(X_all)
r2_train = np.round(polyfit(y_train, y_train_pred),5)
r2_test = np.round(polyfit(y_test, y_test_pred),5)
r2_all = np.round(polyfit(y_all, y_all_pred),5)
print(r2_train, r2_test, r2_all)

0.65624 0.67164 0.66


In [15]:
# Предсказываем значения для тестовой выборки
y_test_main = lng.predict(X_test_main) 

In [16]:
# Записываем результат в файл
resalt = pd.read_csv('submission_example.csv')
resalt['mean_exam_points'] = y_test_main
resalt.to_csv('submission.csv', index=False)

In [17]:
r2_test = np.round(polyfit(y_test, y_test_pred),5)
r2_test

0.67164